In [ ]:
knitr::opts_chunk$set(
  collapse = TRUE,
  comment = "#>", fig.align = "center",
  eval = FALSE
)

In [ ]:
# Install Google Colab dependencies
# Note: this can take 30+ minutes (many of the dependencies include C++ code, which needs to be compiled)

# First install `sf`, `ragg` and `textshaping` and their system dependencies:
system("apt-get -y update && apt-get install -y  libudunits2-dev libgdal-dev libgeos-dev libproj-dev libharfbuzz-dev libfribidi-dev")
install.packages("sf")
install.packages("textshaping")
install.packages("ragg")

# Install system dependencies of some other R packages that Voyager either imports or suggests:
system("apt-get install -y libfribidi-dev libcairo2-dev libmagick++-dev")

# Install Voyager from Bioconductor:
install.packages("BiocManager")
BiocManager::install(version = "3.17", ask = FALSE, update = FALSE, Ncpus = 2)
BiocManager::install("scater")
system.time(
  BiocManager::install("Voyager", dependencies = TRUE, Ncpus = 2, update = FALSE)
)

packageVersion("Voyager")

For website when we submit the new Voyager paper:
1. mouse OB and muscle for Visium*
2. Xenium v2 (this dataset)*
3. MERFISH*
4. VisiumHD*
5. Slide-seq v2
6. CosMX
7. seqFISH
8. CODEX
9. both Visium and metabolomics
10. non-spatial, check in with Sina for updates

Spatial methods for website:
1. Univariate

# Introduction

Xenium is a new technology from 10X genomics for single cell resolution smFISH based spatial transcriptomics. The dataset used here is from adult human pancreatic cancer and processed with Xenium Onboarding Analysis (XOA) v2. It was originally downloaded [here](https://cf.10xgenomics.com/samples/xenium/2.0.0/Xenium_V1_human_Pancreas_FFPE/Xenium_V1_human_Pancreas_FFPE_outs.zip). In the version of the data used here, the large Zarr files have been removed because they're not used here anyway and to speed up download.

Here we load the packages used in this vignette.

In [ ]:
library(Voyager)
library(SFEData) 
library(SingleCellExperiment)
library(SpatialExperiment)
library(SpatialFeatureExperiment)
library(ggplot2)
library(ggforce)
library(stringr)
library(scater) 
library(scuttle)
library(scales)
library(BiocParallel)
library(BiocSingular)
library(bluster)
library(scran)
library(patchwork)
library(RBioFormats)
library(fs)
library(sf)
library(arrow)
library(dplyr)
library(tidyr)
library(BiocNeighbors)
theme_set(theme_bw())
options(timeout = Inf)

In [ ]:
base <- "/Volumes/Archives"
if (!dir.exists(file.path(base, "xenium2_pancreas"))) {
    download.file("https://caltech.box.com/shared/static/6yvpahb97dgp4y2gx7oqjktom9a7qp3o",
                  destfile = file.path(base, "xenium2_pancreas.tar.gz"))
    untar(file.path(base, "xenium2_pancreas.tar.gz"), exdir = base)
}
fn <- file.path(base, "xenium2_pancreas")

This should be the standard XOA v2 output file structure (excluding the Zarr files)

In [ ]:
dir_tree(fn)

The gene count matrix is in the `cell_feature_matrix` h5 or tar.gz file. The cell metadata is in the `cells.csv.gz` or `cells.parquet` file. The cell segmentation polygon vertices are in the `cell_boundaries.csv.gz` or `parquet` file. The nucleus segmentation polygon vertices are in the `nucleus_boundaries.csv.gz` or `parquet` files. The transcript spot coordinates are in the `transcripts.csv.gz` or `parquet` file. The images for different histological stains are in the `morphology_focus` directory. These are the files relevant to the SFE objects; see [10X documentation](https://www.10xgenomics.com/support/software/xenium-onboard-analysis/latest/analysis/xoa-output-at-a-glance) for what the other files are.

The `readXenium()` function in `SpatialFeatureExperiment` reads XOA v1 and v2 outputs into R.

In [ ]:
# Annoying RBioFormats bug
try(sfe <- readXenium(fn))
# Use gene symbols as row names but Ensembl IDs are still in rowData(sfe)
(sfe <- readXenium(fn, row.names = "symbol"))

There are 140702 cells in this dataset, a little more than in the CosMX dataset.

This is what the tissue, with the cell outlines, looks like

In [ ]:
plotGeometry(sfe, "cellSeg")

Plot the nuclei

In [ ]:
plotGeometry(sfe, "nucSeg")

Plot cell density in space

In [ ]:
plotCellBin2D(sfe, binwidth = 50)

There are clearly two different regions, one with high cell density, the other with lower density. They also seem to have very different spatial structures.

Plot total transcript density

In [ ]:
tx <- read_parquet("/Volumes/Archives/xenium2_pancreas/transcripts.parquet")

In [ ]:
ggplot(tx, aes(x_location, -y_location)) +
    geom_bin2d(binwidth = 50) +
    scale_fill_distiller(palette = "Blues", direction = 1) +
    coord_equal() +
    labs(x = NULL, y = NULL)

Cool, so the region with higher cell density also has higher transcript density.

Plot the images; the image has 4 channels, which are DAPI, ATP1A1/CD45/E-Cadherin, 18S, and AlphaSMA/Vimentin, in this order. The images are pyramids with multiple resolutions; some applications would not require the highest resolution. The file for each channel is around 370 MB. Only the metadata is read in R and only the relevant portion of the image at the highest resolution necessary is loaded into memory when needed, say when plotting. 

In [ ]:
getImg(sfe)

There are 4 image files, each for one channel, though the XML metadata of the files connect them together so BioFormats treats them as one file.

In [ ]:
dir_tree(dirname(imgSource(getImg(sfe))))

The image can be plotted with `plotImage()` in `Voyager`, but only up to 3 channels can be ploted at the same time. Say plot 18S (red), ATP1A1/CD45/E-Cadherin (green), and DAPI (blue). These are specified in the `channel` argument, using channel indices. When using 3 channels, by default the indices correspond to R, G, and B channels in this order. The `normalize_channels` argument determines if the channels should be normalized individually in case the different channels have different dynamic ranges. We recommend setting it to `TRUE` for fluorescent images where each channel is separated separately using different fluorophores but not the bright field images where the RGB channels are imaged simultaneously.

In [ ]:
plotImage(sfe, image_id = "morphology_focus", channel = 3:1, show_axes = TRUE,
          dark = TRUE, normalize_channels = TRUE)

Zoom into smaller regions, one in the dense region, another in the sparser region

In [ ]:
bbox1 <- c(xmin = 5500, xmax = 6000, ymin = -1250, ymax = -750)
bbox2 <- c(xmin = 1500, xmax = 2000, ymin = -2000, ymax = -1500)

Plot the bounding boxes in the full image

In [ ]:
bboxes_sf <- c(st_as_sfc(st_bbox(bbox1)), st_as_sfc(st_bbox(bbox2)))
plotImage(sfe, image_id = "morphology_focus", channel = 3:1, show_axes = TRUE,
          dark = TRUE, normalize_channels = TRUE) +
    geom_sf(data = bboxes_sf, fill = NA, color = "white")

In [ ]:
plotImage(sfe, image_id = "morphology_focus", channel = 3:1,
          bbox = bbox1, normalize_channels = TRUE)

In [ ]:
plotImage(sfe, image_id = "morphology_focus", channel = 3:1,
          bbox = bbox2, normalize_channels = TRUE)

Since we're using the RGB channels for color, this is not colorblind friendly. Plot the channels separately to be colorblind friendly:

In [ ]:
p1 <- plotImage(sfe, image_id = "morphology_focus", channel = 1,
          bbox = bbox2) +
    ggtitle("DAPI")
p2 <- plotImage(sfe, image_id = "morphology_focus", channel = 2,
          bbox = bbox2) +
    ggtitle("ATP1A1/CD45/E-Cadherin")
p1 + p2

We can also use a different color map for single channels, such as viridis

In [ ]:
plotImage(sfe, image_id = "morphology_focus", channel = 1,
          bbox = bbox2, palette = viridis_pal()(255))

We can also plot the cell and nuclei geometries on top of the images

In [ ]:
plotGeometry(sfe, "cellSeg", fill = FALSE, dark = TRUE,
             image_id = "morphology_focus", 
             channel = 3:1, bbox = bbox1, normalize_channels = TRUE)

In [ ]:
plotGeometry(sfe, "nucSeg", fill = FALSE, dark = TRUE,
             image_id = "morphology_focus", 
             channel = 3:1, bbox = bbox1, normalize_channels = TRUE)

Plot cells and nuclei together; since there're still quite a lot of cells in that bounding box and the plot looks busy, we can use a even smaller bounding box to plus both cells and nuclei

In [ ]:
bbox3 <- c(xmin = 5750, xmax = 6000, ymin = -1100, ymax = -850)
nuc <- st_intersection(st_as_sfc(st_bbox(bbox3)), nucSeg(sfe))
nuc <- nuc - bbox3[c("xmin", "ymin")]

In [ ]:
plotGeometry(sfe, "cellSeg", fill = FALSE, dark = TRUE,
             image_id = "morphology_focus", 
             channel = 3:1, bbox = bbox3, normalize_channels = TRUE) +
    geom_sf(data = nuc, fill = NA, color = "magenta")

# Quality control
## Negative controls
Some of the features are not genes but negative controls

In [ ]:
names(rowData(sfe))

In [ ]:
unique(rowData(sfe)$Type)

According to the Xenium paper [@Janesick2022-rp], there are 3 types of controls:

> 1) probe controls to assess non-specific binding to RNA, 
2) decoding controls to assess misassigned genes, and 
3) genomic DNA (gDNA) controls to ensure the signal is from RNA.

The paper does not explain in detail how those control probes were designed. The negative controls give us a sense of the false positive rate.

In [ ]:
is_blank <- rowData(sfe)$Type == "Unassigned Codeword"
sum(is_blank)

This should be number 1, the probe control

In [ ]:
is_neg <- rowData(sfe)$Type == "Negative Control Probe"
sum(is_neg)

This should be number 2, the decoding control

In [ ]:
is_neg2 <- rowData(sfe)$Type == "Negative Control Codeword"
sum(is_neg2)

Also make an indicator of whether a feature is any sort of negative control

In [ ]:
is_any_neg <- is_blank | is_neg | is_neg2

The `addPerCellQCMetrics()` function in the `scuttle` package can conveniently add transcript counts, proportion of total counts, and number of features detected for any subset of features to the SCE object. Here we do this for the SFE object, as SFE inherits from SCE. 

In [ ]:
sfe <- addPerCellQCMetrics(sfe, subsets = list(unassigned = is_blank,
                                               negProbe = is_neg,
                                               negCodeword = is_neg2,
                                               any_neg = is_any_neg))

In [ ]:
names(colData(sfe))

Next we plot the proportion of transcript counts coming from any negative control. 

In [ ]:
cols_use <- names(colData(sfe))[str_detect(names(colData(sfe)), "_percent$")]
plotColDataHistogram(sfe, cols_use, bins = 100)

The histogram is dominated by the bin at zero and there are some extreme outliers too few to be seen but evident from the scale of the x axis. We also plot the histogram only for cells with at least 1 count from a negative control. The NA's come from cells that got segmented but have no transcripts detected.

In [ ]:
plotColDataHistogram(sfe, cols_use, bins = 100) + 
    scale_x_log10() +
    annotation_logticks(sides = "b")

For the most part cells have less than 10% of spots assigned to the negative controls.

Next we plot the distribution of the number of negative control counts per cell:

In [ ]:
cols_use <- names(colData(sfe))[str_detect(names(colData(sfe)), "_sum$")]
plotColDataHistogram(sfe, cols_use, bins = 20, ncol = 2) +
    # Avoid decimal breaks on x axis unless there're too few breaks
    scale_x_continuous(breaks = scales::breaks_extended(Q = c(1,2,5))) +
    scale_y_log10() +
    annotation_logticks(sides = "l")

The counts are low, mostly zero, but there are some cells with up to 2 counts of all types aggregated. Then the outlier with 30% of counts from negative controls must have very low total real transcript counts to begin with. The negative controls indicates the prevalence of false positives. Here we have only about 2000 out of over 140,000 that have negative control spots detected. About `r format(sum(tx$feature_name %in% rownames(sfe)[is_any_neg])/nrow(tx)*100, digits = 3)`% are negative control spots, so false positive rate is low.

In [ ]:
names(colData(sfe))

Where are cells with higher proportion of negative control spots (i.e. low total counts and have negative control spots detected, >10% here) located in space?

In [ ]:
data("ditto_colors")

In [ ]:
sfe$more_neg <- sfe$subsets_any_neg_percent > 10
plotSpatialFeature(sfe, "subsets_any_neg_sum", colGeometryName = "cellSeg",
                   show_axes = TRUE) +
    geom_sf(data = SpatialFeatureExperiment::centroids(sfe)[sfe$more_neg,],
            size = 0.5, color = ditto_colors[1]) +
    theme(legend.position = "bottom")

Cells that have any negative control spots appear to be randomly distributed in space, but cells that have higher proportion of negative control seem to be few and over-represented in the less dense area, because the proportion is higher when total transcript counts are lower to begin with.

Where are negative control spots located?

In [ ]:
tx |> 
    filter(feature_name %in% rownames(sfe)[is_any_neg]) |> 
    ggplot(aes(x_location, -y_location)) +
    geom_bin2d(binwidth = 50) +
    scale_fill_distiller(palette = "Blues", direction = 1) +
    labs(x = NULL, y = NULL)

Generally there are more negative control spots in the region with higher cell and transcript density, but especially regions with high cell density, which is not surprising. There don't visually seem to be regions with more negative control spots not accounted for by cell and transcript density. In contrast, the first Xenium preview data from 2022 has a region in the top left corner with more negative control probes detected that seems like an artifact (see `JanesickBreastData()`).

## Cells
Some QC metrics are precomputed and are stored in `colData`

In [ ]:
names(colData(sfe))

Since there're more cells, it would be better to plot the tissue larger, so we'll plot the histogram of QC metrics and the spatial plots separately, unlike in the CosMx vignette.

In [ ]:
n_panel <- sum(!is_any_neg)
sfe$nCounts <- colSums(counts(sfe)[!is_any_neg,])
sfe$nGenes <- colSums(counts(sfe)[!is_any_neg,] > 0)
colData(sfe)$nCounts_normed <- sfe$nCounts/n_panel
colData(sfe)$nGenes_normed <- sfe$nGenes/n_panel

Here we divided the total number of transcript molecules detected from genes (`nCounts`) and the number of genes detected (`nGenes`) by the total number of genes probed, so this histogram is comparable to those from other smFISH-based datasets. 

In [ ]:
plotColDataHistogram(sfe, c("nCounts", "nGenes"))

There're weirdly some discrete values of the number of genes detected, which excludes negative controls.

In [ ]:
plotColDataHistogram(sfe, c("nCounts_normed", "nGenes_normed"))

Compared to the [FFPE CosMX non-small cell lung cancer dataset](https://pachterlab.github.io/voyager/articles/vig4_cosmx.html#cells), fewer transcripts per gene on average and a smaller proportion of all genes are detected in this dataset, which is also FFPE. However, this should be interpreted with care, since these two datasets are from different tissues and have different gene panels, so this may or may not indicate that Xenium has better detection efficiency than CosMX. See (I know the two papers) for systematic comparisons of different imaging based technologies with the same tissue.

In [ ]:
plotSpatialFeature(sfe, "nCounts", colGeometryName = "cellSeg")

In [ ]:
plotSpatialFeature(sfe, "nGenes", colGeometryName = "cellSeg")

These plots clearly show that nCounts and nGenes are spatially structured and may be biologically relevant.

A standard examination is to look at the relationship between nCounts and nGenes:

In [ ]:
plotColData(sfe, x="nCounts", y="nGenes", bins = 70) +
    scale_fill_distiller(palette = "Blues", direction = 1)

Here we plot the distribution of cell area

In [ ]:
plotColDataHistogram(sfe, c("cell_area", "nucleus_area"), scales = "free_y")

There's a very long tail. The nuclei are much smaller than the cells.

How is cell area distributed in space?

In [ ]:
plotSpatialFeature(sfe, "cell_area", colGeometryName = "cellSeg", 
                   show_axes = TRUE)

The largest cells tend to be in the sparse area. This may be biological or an artifact of the cell segmentation algorithm or both.

Here the nuclei segmentations are plotted instead of cell segmentation.

In [ ]:
plotSpatialFeature(sfe, "nucleus_area", colGeometryName = "nucSeg",
                   show_axes = TRUE)

Zoom into smaller regions to see the nature of the very large cells

In [ ]:
bbox3 <- c(xmin = 1350, xmax = 1550, ymin = -1750, ymax = -1550)
bbox4 <- c(xmin = 3400, xmax = 3600, ymin = -2900, ymax = -2700)

In [ ]:
nuc <- st_intersection(st_as_sfc(st_bbox(bbox3)), nucSeg(sfe))
nuc <- nuc - bbox3[c("xmin", "ymin")]

In [ ]:
plotGeometry(sfe, "cellSeg", fill = FALSE, dark = TRUE,
             image_id = "morphology_focus", 
             channel = 3:1, bbox = bbox3, normalize_channels = TRUE) +
    geom_sf(data = nuc, fill = NA, color = "magenta")

In [ ]:
nuc <- st_intersection(st_as_sfc(st_bbox(bbox4)), nucSeg(sfe))
nuc <- nuc - bbox4[c("xmin", "ymin")]

In [ ]:
plotGeometry(sfe, "cellSeg", fill = FALSE, dark = TRUE,
             image_id = "morphology_focus", 
             channel = 3:1, bbox = bbox4, normalize_channels = TRUE) +
    geom_sf(data = nuc, fill = NA, color = "magenta")

In most cases, the large cells don't immediately look like artifacts of segmentation, so maybe don't remove them for QC purposes. Meanwhile we see different morphologies in the cells and nuclei which can be interesting to further explore. There are also some cells that don't have nuclei which may have nuclei in a different z-plane or may be false positives, and maybe some false negatives in regions with fluorescence and seemingly nuclei but not cell segmentation.

Next we calculate the proportion of cell in this z-plane taken up by the nucleus, and examine the distribution:

In [ ]:
colData(sfe)$prop_nuc <- sfe$nucleus_area / sfe$cell_area

In [ ]:
summary(sfe$prop_nuc)

In [ ]:
plotColDataHistogram(sfe, "prop_nuc")

The NA's are cells that don't have nuclei detected. There are also nearly 2500 cells entirely taken up the their nuclei, which may be due to cell type or false negative in segmenting the rest of the cell.

Here we plot the nuclei proportion in space:

In [ ]:
plotSpatialFeature(sfe, "prop_nuc", colGeometryName = "cellSeg")

Cells in some histological regions have larger proportions occupied by the nuclei. It is interesting to check, controlling for cell type, how cell area, nucleus area, and the proportion of cell occupied by nucleus relate to gene expression. However, a problem in performing such an analysis is that cell segmentation is only available for one z-plane here and these areas also relate to where this z-plane intersects each cell. 

Does cell area relate to `nCounts`, nucleus area, proportion of area taken up by nucleus, and so on? Here we plot how each pair of variables relate to each other in a matrix plot using the [`ggforce`](https://ggforce.data-imaginist.com/) package.

In [ ]:
cols_use <- c("nCounts", "nGenes", "cell_area", "nucleus_area", "prop_nuc")
df <- colData(sfe)[,cols_use] |> as.data.frame()

In [ ]:
ggplot(df) +
    geom_bin2d(aes(x = .panel_x, y = .panel_y), bins = 30) +
    geom_autodensity(fill = "gray90", color = "gray70", linewidth = 0.2) +
    facet_matrix(vars(tidyselect::everything()), layer.diag = 2) +
    scale_fill_distiller(palette = "Blues", direction = 1)

Cool, so, generally, `nCounts`, `nGenes`, cell area, and nucleus area positively correlate with each other, but the proportion of cell area taken up by the nucleus doesn't seem to correlate with the other variables.

In [ ]:
sfe$has_nuclei <- !is.na(sfe$nucleus_area)
plotColData(sfe, y = "has_nuclei", x = "cell_area", 
            point_fun = function(...) list())

It seems that cells that don't have nuclei detected tend to be smaller, probably because less of them are captured in this tissue section when their nuclei are not in this section. Where are the cells without nuclei distributed in space?

In [ ]:
plotSpatialFeature(sfe, "has_nuclei", colGeometryName = "cellSeg",
                   show_axes = TRUE) +
    # To highlight the cells that don't have nuclei
    geom_sf(data = SpatialFeatureExperiment::centroids(sfe)[!sfe$has_nuclei,], 
            color = ditto_colors[1], size = 0.3)

It seems that there are more cells without nuclei in regions with higher cell density; would be interesting to do spatial point process analysis. Zoom into a smaller region to inspect, especially the cells outside the main piece of tissue:

In [ ]:
bbox5 <- c(xmin = 6400, xmax = 6800, ymin = -300, ymax = -50)
bbox6 <- c(xmin = 600, xmax = 1000, ymin = -600, ymax = -300)
bbox7 <- c(xmin = 6800, xmax = 7200, ymin = -2900, ymax = -2500)

In [ ]:
nuc <- st_intersection(st_as_sfc(st_bbox(bbox5)), nucSeg(sfe))
nuc <- nuc - bbox5[c("xmin", "ymin")]

In [ ]:
plotGeometry(sfe, "cellSeg", fill = FALSE, dark = TRUE,
             image_id = "morphology_focus", show_axes = TRUE,
             channel = 3:1, bbox = bbox5, normalize_channels = TRUE) +
    geom_sf(data = nuc, fill = NA, color = "magenta")

In [ ]:
nuc <- st_intersection(st_as_sfc(st_bbox(bbox6)), nucSeg(sfe))
nuc <- nuc - bbox6[c("xmin", "ymin")]

In [ ]:
plotGeometry(sfe, "cellSeg", fill = FALSE, dark = TRUE,
             image_id = "morphology_focus", show_axes = TRUE,
             channel = 3:1, bbox = bbox6, normalize_channels = TRUE) +
    geom_sf(data = nuc, fill = NA, color = "magenta")

In [ ]:
nuc <- st_intersection(st_as_sfc(st_bbox(bbox7)), nucSeg(sfe))
nuc <- nuc - bbox7[c("xmin", "ymin")]

In [ ]:
plotGeometry(sfe, "cellSeg", fill = FALSE, dark = TRUE,
             image_id = "morphology_focus", show_axes = TRUE,
             channel = 3:1, bbox = bbox7, normalize_channels = TRUE) +
    geom_sf(data = nuc, fill = NA, color = "magenta")

Here in QC we are looking for low quality cells. From what we've explored so far, cells with negative control counts and exceptionally large cells might not be suspect, but cells far outside the tissue are suspect. For some types of spatial neighborhood graphs, such as the k nearest neighbor graph, these cells will also affect spatial analysis. So here we remove those cells. Here we compute a k nearest neighbor graph with k = 5 and remove cells whith neighbors that are too far away.

In [ ]:
g <- findKNN(spatialCoords(sfe)[,1:2], k = 5, BNPARAM = AnnoyParam())

In [ ]:
max_dist <- rowMaxs(g$distance)
data.frame(max_dist = max_dist) |> 
    ggplot(aes(max_dist)) +
    geom_histogram(bins = 100) +
    scale_y_continuous(transform = "log1p") +
    scale_x_continuous(breaks = breaks_pretty()) +
    annotation_logticks(sides = "l")

Also look at minimum distance between the neighbors to find outlying single cells that may be less far from the tissue

In [ ]:
min_dist <- rowMins(g$distance)
data.frame(min_dist = min_dist) |> 
    ggplot(aes(min_dist)) +
    geom_histogram(bins = 100) +
    scale_y_continuous(transform = "log1p") +
    scale_x_continuous(breaks = breaks_pretty()) +
    annotation_logticks(sides = "l")

There's a long tail, which must be the small clusters of cells away from the tissue. Say use a cutoff of 60 microns, and see where those cells are.

In [ ]:
sfe$main_tissue <- !(max_dist > 60 | min_dist > 50)

In [ ]:
plotSpatialFeature(sfe, "main_tissue", colGeometryName = "cellSeg",
                   show_axes = TRUE) +
    # To highlight the outliers
    geom_sf(data = SpatialFeatureExperiment::centroids(sfe)[!sfe$main_tissue,], 
            color = ditto_colors[1], size = 0.3)

To be more fastidious, I can also compare the max neighbor distance of each cell to those of its neighbors in order to account for cell density in different parts of the tissue, and to extract data from the image to remove cells that lack fluorescent signals. Here we remove the cells away from the main tissue, cells that have very low total gene expression, cells that are too small to make sense, and genes that are not detected.

In [ ]:
plotColDataHistogram(sfe, c("nCounts", "cell_area")) +
    scale_x_log10() +
    annotation_logticks(sides = "b")

The x axis is log transformed to better see the small values, and there's no obvious outlier. Say use an arbitrary cutoff of 5 for nCounts

In [ ]:
sfe <- sfe[,sfe$main_tissue & sfe$nCounts > 5]
sfe <- sfe[rowSums(counts(sfe)) > 0,]
dim(sfe)

## Genes
Here we look at the mean and variance of each gene

In [ ]:
rowData(sfe)$means <- rowMeans(counts(sfe))
rowData(sfe)$vars <- rowVars(counts(sfe))

Real genes generally have higher mean expression across cells than negative controls. This is the mean of each gene across cells.

In [ ]:
rowData(sfe)$is_neg <- rowData(sfe)$Type != "Gene Expression"
plotRowData(sfe, x = "means", y = "is_neg") +
    scale_y_log10() +
    annotation_logticks(sides = "b")

Here the real genes and negative controls are plotted in different colors

In [ ]:
plotRowData(sfe, x="means", y="vars", color_by = "is_neg") +
    geom_abline(slope = 1, intercept = 0, color = "red") +
    scale_x_log10() + scale_y_log10() +
    annotation_logticks() +
    coord_equal() +
    labs(color = "Negative control")

The red line $y = x$ is expected if the data follows a Poisson distribution. Negative controls and real genes form mostly separate clusters. Negative controls stick close to the line, while real genes are overdispersed. Unlike in the [CosMX dataset](https://pachterlab.github.io/voyager/articles/vig4_cosmx.html#genes), the negative controls don't seem overdispersed. Overdispersion in gene expression can not only be caused by transcription bursts and cell type heterogeneity, but also by both positive and negative spatial autocorrelation [@Chun2018-ar; @Griffith2011-lx].

# Spatial autocorrelation of QC metrics

There's a sparse and a dense region. This poses the question of what type of neighborhood graph to use, e.g. it is conceivable that cells in the sparse region should just be singletons. Furthermore, it is unclear what the length scale of their influence might be. It might depend on the cell type and how contact and secreted signals are used in the cell type, and length scale of the influence. If k nearest neighbors are used, then the neighbors in the dense region are much closer together than those in the sparse region. If distance based neighbors are used, then cells in the dense region will have more neighbors than cells in the sparse region, and the sparse region can break into multiple compartments if the distance cutoff is not long enough. 

For the purpose of demonstration, we use k nearest neighbors with $k = 5$, with inverse distance weighting. Note that using more neighbors leads to longer computation time of spatial autocorrelation metrics.

In [ ]:
colGraph(sfe, "knn5") <- findSpatialNeighbors(sfe, method = "knearneigh", 
                                              dist_type = "idw", k = 5, 
                                              style = "W")

I think it makes sense to replace NA with 0 for nucleus area here though generally 0 is not an appropriate substitute for NA. 

In [ ]:
sfe$nucleus_area <- ifelse(is.na(sfe$nucleus_area), 0, sfe$nucleus_area)

In [ ]:
sfe <- colDataMoransI(sfe, c("nCounts", "nGenes", "cell_area", "nucleus_area"),
                      colGraphName = "knn5")

In [ ]:
colFeatureData(sfe)[c("nCounts", "nGenes", "cell_area", "nucleus_area"),]

Global Moran's I indicates positive spatial autocorrelation. Here global spatial autocorrelation for these QC metrics is moderate or weak, at least at the level of single cells using a k nearest neighbor graph with k = 5. From earlier plots of these metrics, there may be stronger spatial autocorrelation at a longer length scale. As the strength of spatial autocorrelation can vary spatially, we also run local Moran's I.

In [ ]:
sfe <- colDataUnivariate(sfe, type = "localmoran", 
                         features = c("nCounts", "nGenes", "cell_area", 
                                      "nucleus_area"),
                         colGraphName = "knn5", BPPARAM = MulticoreParam(2))

The `pointsize` argument adjusts the point size in `scattermore`. The default is 0, meaning single pixels, but since the cells in the sparse region are hard to see that way, we increase `pointsize`. We would still plot the polygons in larger single panel plots, but use `scattermore` in multi-panel plots where the polygons in each panel are invisible anyway due to the small size to save some time.

In [ ]:
plotLocalResult(sfe, "localmoran",
                features = c("nCounts", "nGenes", "cell_area", "nucleus_area"),
                colGeometryName = "centroids", scattermore = TRUE,
                divergent = TRUE, diverge_center = 0, pointsize = 1.5,
                ncol = 1)

It seems that the glandular areas tend to be more homogeneous, greatly contributing to the global Moran's I value, but most of the tissue is not very spatially autocorrelated for these metrics. The $I_i$ value is not the only thing computed in local Moran's I. P-values and types of neighborhoods are also computed. "Type of neighborhood" include cells with low values and neighbors with low values (low-low), low-high, high-high, and high-low, and "low" and "high" are relative to the mean. One can make a scatter plot with the value at each cell on the x axis and spatially weighted average of the cell's neighbors on the spatial neighborhood graph on the y axis; this is the Moran scatter plot. These neighborhood types are quadrants on the plot. However, when the value doesn't deviate much from the mean, the neighborhood type might not mean much, but it does seem to highlight some tissue structure. Another caveat is that the mean is from the entire section, while from tissue morphology, there clearly are two very different regions within the section. It would be cool to have a method to better determine areas in the tissue for spatial analysis that can best answer the questions of interest. It might make sense to analyze the sparse and dense regions separately at some point.

In [ ]:
localResultAttrs(sfe, "localmoran", "nCounts")

In [ ]:
plotLocalResult(sfe, "localmoran", attribute = "pysal",
                features = c("nCounts", "nGenes", "cell_area", "nucleus_area"),
                colGeometryName = "centroids", scattermore = TRUE, pointsize = 1.5,
                ncol = 1, size = 3)

It's hard to see without zooming in.

In [ ]:
plotLocalResult(sfe, "localmoran",
                features = c("nCounts", "nGenes", "cell_area", "nucleus_area"),
                colGeometryName = "cellSeg", divergent = TRUE, diverge_center = 0,
                ncol = 2, bbox = bbox1)

In [ ]:
plotLocalResult(sfe, "localmoran", attribute = "pysal",
                features = c("nCounts", "nGenes", "cell_area", "nucleus_area"),
                colGeometryName = "cellSeg", bbox = bbox1, ncol = 2)

In [ ]:
plotLocalResult(sfe, "localmoran", attribute = "-log10p_adj", 
                features = c("nCounts", "nGenes", "cell_area", "nucleus_area"),
                colGeometryName = "cellSeg", divergent = TRUE, 
                diverge_center = -log10(0.05),
                ncol = 2, bbox = bbox1)

Here the center of the divergent palette is at $-\mathrm{log}_{10}0.05$ and the p-values have been corrected for multiple testing (see `spdep::p.adjustSP()`). Warm color indicates "significant" and cool color indicates "not significant" based on this conventional threshold. Here in this dense region, for these metrics, local Moran's I is generally not significant.

In [ ]:
plotLocalResult(sfe, "localmoran",
                features = c("nCounts", "nGenes", "cell_area", "nucleus_area"),
                colGeometryName = "cellSeg", divergent = TRUE, diverge_center = 0,
                ncol = 2, bbox = bbox2)

In [ ]:
plotLocalResult(sfe, "localmoran", attribute = "pysal",
                features = c("nCounts", "nGenes", "cell_area", "nucleus_area"),
                colGeometryName = "cellSeg", bbox = bbox2, ncol = 2)

In [ ]:
plotLocalResult(sfe, "localmoran", attribute = "-log10p_adj", 
                features = c("nCounts", "nGenes", "cell_area", "nucleus_area"),
                colGeometryName = "cellSeg", divergent = TRUE, 
                diverge_center = -log10(0.05),
                ncol = 2, bbox = bbox2)

In this sparser region, there're more areas where local Moran's I is significant for these metrics. It seems that locally we can have a mixture of positive and negative spatial autocorrelation, though for the most part local Moran's I is not statistically significant and the significant parts are in the high-high regions. What to make of this? 

Moran plot for nCounts

In [ ]:
sfe <- colDataUnivariate(sfe, "moran.plot", "nCounts", colGraphName = "knn5")

In [ ]:
moranPlot(sfe, "nCounts", binned = TRUE, plot_influential = FALSE, bins = c(150, 60)) 

There are no obvious clusters here based on the 2D histogram showing point density in the scatter plot. With W style (the adjacency matrix with edge weights have row sums of 1), the slope of the least square line fitted to the scatter plot is Moran's I [@Anselin1996-mo].

# Moran's I

By default, for gene expression, the log normalized counts are used in spatial autocorrelation metrics because many statistical methods were developed for normally distributed data and there can be technical artifacts affecting the raw gene counts per cell, so before running Moran's I, we normalize the data. In the single cell tradition, total counts (nCounts) is treated like a technical artifact to be normalized away, which makes sense given the transcript capture and PCR amplification steps in the single cell RNA-seq protocol. However, as seen above, nCounts clearly has biologically relevant spatial structure, and imaging based technologies don't have those steps. Furthermore, in imaging based technologies, typically a curated gene panel of a few hundred genes targeting certain cell types of interest is used, while scRNA-seq is usually untargeted and transcriptome wide. Due to the skewed gene panel, nCounts is biologically skewed as well so using it to normalize data can lead to false negatives in finding spatially variable genes and differential expression [@Atta2024-tl]. 

Sometimes cell area is used when normalizing imaging based data, which makes sense when larger cells have more room for more transcripts and as seen above in the matrix plot, cell area does positively correlate with nCounts. However, as seen in the earlier cell level QC plots, cell size is also spatially structured and seemingly biologically relevant because cells of some types tend to be larger than cells of some other types, though its global Moran's I is a bit weaker than that of nCounts. Often some cells are larger because more of them get captured in this section and some cells are smaller because less of them are in this section, so there still is a technical component. In [@Atta2024-tl], normalizing by cell area and not normalizing are also examined; here SVG and DE results are more comparable to when using a more representative gene panel, with fewer false positives and false negatives compared to normalizing by nCounts. Due to the technical effect of partial capture of cells in the section, [@Atta2024-tl] recommends normalizing with cell area if it's reliable. While cell area from XOA v1 with cell segmentations that look like Voroni tessellation doesn't seem reliable, with multiple fluorescent stains in XOA v2, it seems much better, so we'll normalize with cell area here while noting the caveat that cell area can be biologically relevant.

In [ ]:
sfe <- logNormCounts(sfe, size.factor = sfe$cell_area)

Use more cores if available to speed this up. It's slower here because the gene count matrix is actually on disk thanks to `DelayedArray` and not loaded into memory.

In [ ]:
system.time(
    sfe <- runMoransI(sfe, colGraphName = "knn5", BPPARAM = MulticoreParam(2))
)

In [ ]:
plotRowData(sfe, x = "moran_sample01", y = "is_neg")

As expected, generally the negative controls are tightly clustered around 0, while the real genes have positive Moran's I, which means there is generally no technical artifact spatial trend. 

What are the genes with the highest Moran's I?

In [ ]:
top_moran <- rownames(sfe)[order(rowData(sfe)$moran_sample01, decreasing = TRUE)[1:4]]
plotSpatialFeature(sfe, top_moran, colGeometryName = "centroids",
                   scattermore = TRUE, pointsize = 1.5, ncol = 1)

AMY2A is amylase alpha 2A, involved in breaking down starch. INS is insulin. GCG is glucagon. GATM is glycine amidinotransferase involved in creatine biosynthesis. So the dense area have exocrine acini making digestive enzymes and the endocrine Langerhan's islets making insulin and glucagon. These are very spatially structured. Zoom into a smaller area

In [ ]:
bbox8 <- c(xmin = 4950, xmax = 5450, ymin = -1000, ymax = -500)

In [ ]:
plotSpatialFeature(sfe, top_moran, colGeometryName = "cellSeg", ncol = 2,
                   bbox = bbox8)

In [ ]:
top_moran2 <- rownames(sfe)[order(rowData(sfe)$moran_sample01, decreasing = TRUE)[5:8]]
plotSpatialFeature(sfe, top_moran2, colGeometryName = "centroids",
                   scattermore = TRUE, pointsize = 1.5, ncol = 1)

GPRC5A is G protein-coupled receptor class C group 5 member A, which may play a role in embryonic development and epithelial cell differentiation. SST is somatostatin, which is a regulator of the endocrine system. CFTR encodes the chlorine ion channel whose mutation causes cystic fibrosis. DMBT1 stands for deleted in malignant brain tumors 1. "Loss of sequences from human chromosome 10q has been associated with the progression of human cancers", according to RefSeq. So the sparse region is probably cancerous. Zoom into a smaller area

In [ ]:
plotSpatialFeature(sfe, top_moran2, colGeometryName = "cellSeg", ncol = 2,
                   bbox = bbox2)

How does Moran's I relate to gene expression level?

In [ ]:
plotRowData(sfe, x = "means", y = "moran_sample01", color_by = "is_neg") +
    scale_x_log10() + annotation_logticks(sides = "b")

Very highly expressed genes have higher Moran's I, but there are some less expressed genes with higher Moran's I as well.

# Non-spatial dimension reduction and clustering

Here we run non-spatial PCA as for scRNA-seq data

In [ ]:
set.seed(29)
sfe <- runPCA(sfe, ncomponents = 30, scale = TRUE, BSPARAM = IrlbaParam(),
              subset_row = !rowData(sfe)$is_neg)

In [ ]:
ElbowPlot(sfe, ndims = 30)

In [ ]:
plotDimLoadings(sfe, dims = 1:8)

In [ ]:
spatialReducedDim(sfe, "PCA", 4, colGeometryName = "centroids", divergent = TRUE,
                  diverge_center = 0, ncol = 1, scattermore = TRUE, pointsize = 1.5)

In [ ]:
spatialReducedDim(sfe, "PCA", components = 5:8, colGeometryName = "centroids", 
                  divergent = TRUE, diverge_center = 0, 
                  ncol = 1, scattermore = TRUE, pointsize = 1.5)

While spatial region is not explicitly used, the PC's highlight spatial regions due to spatial autocorrelation in gene expression and histological regions with different cell types.

Non-spatial clustering and locating the clusters in space

In [ ]:
colData(sfe)$cluster <- clusterRows(reducedDim(sfe, "PCA")[,1:15],
                                    BLUSPARAM = SNNGraphParam(
                                        cluster.fun = "leiden",
                                        cluster.args = list(
                                            resolution_parameter = 0.5,
                                            objective_function = "modularity")))

Now the `scater` can also rasterize the plots with lots of points with the `rasterise` argument, but with a different mechanism from `scattermore` that requires more system dependencies. 

In [ ]:
plotPCA(sfe, ncomponents = 4, colour_by = "cluster", scattermore = TRUE)

Plot the location of the clusters in space

In [ ]:
plotSpatialFeature(sfe, "cluster", colGeometryName = "centroids", scattermore = TRUE,
                   pointsize = 1.2, size = 3)

# Differential expression
Cluster marker genes are found with Wilcoxon rank sum test as commonly done for scRNA-seq.

In [ ]:
markers <- findMarkers(sfe, groups = colData(sfe)$cluster,
                       test.type = "wilcox", pval.type = "all", direction = "up")

It's already sorted by p-values:

In [ ]:
markers[[6]]

The code below extracts the significant markers for each cluster:

In [ ]:
genes_use <- vapply(markers, function(x) rownames(x)[1], FUN.VALUE = character(1))
plotExpression(sfe, genes_use, x = "cluster", point_fun = function(...) list())

This allows for plotting more top marker genes in a heatmap:

In [ ]:
genes_use2 <- unique(unlist(lapply(markers, function(x) rownames(x)[1:5])))
plotGroupedHeatmap(sfe, genes_use2, group = "cluster", colour = scales::viridis_pal()(100))

# Local spatial statistics of marker genes

First we plot those genes in space as a reference

In [ ]:
plotSpatialFeature(sfe, genes_use, colGeometryName = "centroids", ncol = 2,
                   pointsize = 0, scattermore = TRUE)

Global Moran's I of these marker genes is shown below:

In [ ]:
setNames(rowData(sfe)[genes_use, "moran_sample01"], genes_use)

All these marker genes have positive spatial autocorrelation, but some stronger than others.

Local Moran's I of these marker genes is shown below:

In [ ]:
sfe <- runUnivariate(sfe, "localmoran", features = genes_use, colGraphName = "knn5",
                     BPPARAM = MulticoreParam(2))

In [ ]:
plotLocalResult(sfe, "localmoran", features = genes_use, 
                colGeometryName = "centroids", ncol = 2, divergent = TRUE,
                diverge_center = 0, scattermore = TRUE, pointsize = 0)

It seems that some histological regions tend to be more spatially homogenous in gene expression than others. The epithelial region tends to be more homogenous. For some genes, regions with higher expression also have higher local Moran's I, such as FOXA1 and GATA3, while for some genes, this is not the case, such as FGL2 and LUM. 

Finally, we assess local spatial heteroscdasticity (LOSH) for these marker genes to find local heterogeneity:

In [ ]:
sfe <- runUnivariate(sfe, "LOSH", features = genes_use, colGraphName = "knn5",
                     BPPARAM = MulticoreParam(2))

In [ ]:
plotLocalResult(sfe, "LOSH", features = genes_use, 
                colGeometryName = "centroids", ncol = 2, scattermore = TRUE, 
                pointsize = 0)

Again, just like in the [CosMX dataset](https://pachterlab.github.io/voyager/articles/vig4_cosmx.html#local-spatial-statistics-of-marker-genes), LOSH is higher where the gene is more highly expressed in some (e.g. CD3E, LUM, TENT5C) but not all cases (e.g. FOXA1, GATA3). This may be due to spatial distribution of different cell types.

# Explore multiple length scales
Here we perform spatial binning of the transcript spots with bins of different sizes to see how the spatial statistics change over length scales.

# Session info

In [ ]:
sessionInfo()

# References